### Data Cleaning Player Stats file 

In [94]:
import pandas as pd
import numpy as np
from pandas_profiling import ProfileReport
import os
from pandas import DataFrame
import datetime
import matplotlib.pyplot as plt

In [2]:
# Reading player statistics file
df_pstats = pd.read_csv('playerstats.csv')

/var/folders/cp/9jtgx6ps6ldg63161fmh4z840000gn/T/ipykernel_10885/1619808696.py:2: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  df_pstats = pd.read_csv('playerstats.csv')


In [3]:
df_pstats.shape

(32200183, 20)

In [4]:
df_pstats.head()

,game_id,game_start_time,game_duration_minutes,game_validity,game_featuremod_name,game_total_players,map_id,map_name,map_width,map_height,player_id,player_login,player_faction,player_team,player_starting_spot,player_result,player_is_computer,before_rating,after_rating,leaderboard_ids
0,17405720,2022-06-30 23:59:58 UTC,30.7,UNKNOWN_RESULT,FAF,1,21833.0,Hold the line,2048.0,2048.0,431007,Walgreens,UEF,3,1,UNKNOWN,False,0.0,NaN,NaN
1,17405711,2022-06-30 23:59:44 UTC,30.2,COOP_NOT_RANKED,Coop,1,NaN,NaN,NaN,NaN,96808,Starcruiser,Cybran,1,1,UNKNOWN,False,0.0,NaN,NaN
2,17405650,2022-06-30 23:58:34 UTC,48.0,VALID,FAF,12,14570.0,Desert Gap,1024.0,1024.0,303409,ourubapu,UEF,3,3,DEFEAT,False,163.3,159.7,1
3,17405650,2022-06-30 23:58:34 UTC,48.0,VALID,FAF,12,14570.0,Desert Gap,1024.0,1024.0,436692,Enzord_Rocka,UEF,2,11,DEFEAT,False,686.7,692.4,1
4,17405650,2022-06-30 23:58:34 UTC,48.0,VALID,FAF,12,14570.0,Desert Gap,1024.0,1024.0,17943,Defiant,UEF,3,5,DEFEAT,False,1082.8,1078.0,1


In [5]:
# Dropping unnecessary columns
to_drop_play = ['leaderboard_ids','game_duration_minutes','player_starting_spot']
df_pstats.drop(to_drop_play,axis = 1, inplace= True)

In [6]:
# Renaming columns
df_pstats.rename(columns={'player_faction':'faction'},inplace=True)

In [7]:
# Finding nulls
df_pstats.isnull().sum()

game_id                        0
game_start_time                0
game_validity                  0
game_featuremod_name           0
game_total_players             0
map_id                   2564564
map_name                 2564564
map_width                2564564
map_height               2564564
player_id                      0
player_login                 557
faction                        0
player_team                    0
player_result                  0
player_is_computer             0
before_rating                  0
after_rating            18327390
dtype: int64

map_id : -1
map_name : Unknown
map_width : -1
map_height : -1
player_login : Unknown
after_rating : 99999999

In [8]:
#setting unknown mapids to 99999999
-1 in df_pstats['map_id']

False

In [9]:
df_pstats["map_id"].replace(to_replace=np.nan,value=-1,inplace=True)

In [10]:
"Unknown" in df_pstats['map_name']

False

In [11]:
df_pstats["map_name"].replace(to_replace=np.nan,value="Unknown",inplace=True)

In [12]:
-1 in df_pstats['map_width']

False

In [13]:
df_pstats["map_width"].replace(to_replace=np.nan,value=-1,inplace=True)

In [14]:
-1 in df_pstats['map_height']

False

In [15]:
df_pstats["map_height"].replace(to_replace=np.nan,value=-1,inplace=True)

In [16]:
"Unknown" in df_pstats['player_login']

False

In [17]:
df_pstats["player_login"].replace(to_replace=np.nan,value="Unknown",inplace=True)

In [18]:
99999999 in df_pstats['after_rating']

False

In [19]:
df_pstats["after_rating"].replace(to_replace=np.nan,value=99999999,inplace=True)

In [20]:
df_pstats['timezone'] = df_pstats['game_start_time'].apply(lambda x : x.split(" ")[-1])

In [21]:
df_pstats['start_date'] = df_pstats['game_start_time'].apply(lambda x : x.split(" ")[0])

In [22]:
df_pstats['start_time'] = df_pstats['game_start_time'].apply(lambda x : x.split(" ")[1])

In [23]:
df_pstats['start_year'] = df_pstats['game_start_time'].apply(lambda x : x.split("-")[0])

In [24]:
df_pstats['start_date'] = pd.to_datetime(df_pstats['start_date'],errors='coerce')

In [25]:
df_pstats["start_month"] = df_pstats["start_date"].dt.month_name()

In [26]:
df_pstats.head()

,game_id,game_start_time,game_validity,game_featuremod_name,game_total_players,map_id,map_name,map_width,map_height,player_id,...,player_team,player_result,player_is_computer,before_rating,after_rating,timezone,start_date,start_time,start_year,start_month
0,17405720,2022-06-30 23:59:58 UTC,UNKNOWN_RESULT,FAF,1,21833.0,Hold the line,2048.0,2048.0,431007,...,3,UNKNOWN,False,0.0,99999999.0,UTC,2022-06-30,23:59:58,2022,June
1,17405711,2022-06-30 23:59:44 UTC,COOP_NOT_RANKED,Coop,1,-1.0,Unknown,-1.0,-1.0,96808,...,1,UNKNOWN,False,0.0,99999999.0,UTC,2022-06-30,23:59:44,2022,June
2,17405650,2022-06-30 23:58:34 UTC,VALID,FAF,12,14570.0,Desert Gap,1024.0,1024.0,303409,...,3,DEFEAT,False,163.3,159.7,UTC,2022-06-30,23:58:34,2022,June
3,17405650,2022-06-30 23:58:34 UTC,VALID,FAF,12,14570.0,Desert Gap,1024.0,1024.0,436692,...,2,DEFEAT,False,686.7,692.4,UTC,2022-06-30,23:58:34,2022,June
4,17405650,2022-06-30 23:58:34 UTC,VALID,FAF,12,14570.0,Desert Gap,1024.0,1024.0,17943,...,3,DEFEAT,False,1082.8,1078.0,UTC,2022-06-30,23:58:34,2022,June


In [27]:
# Finding nulls
df_pstats.isnull().sum()

game_id                 0
game_start_time         0
game_validity           0
game_featuremod_name    0
game_total_players      0
map_id                  0
map_name                0
map_width               0
map_height              0
player_id               0
player_login            0
faction                 0
player_team             0
player_result           0
player_is_computer      0
before_rating           0
after_rating            0
timezone                0
start_date              0
start_time              0
start_year              0
start_month             0
dtype: int64

In [28]:
# Dropping unnecessary columns
to_drop_play = ['game_start_time']
df_pstats.drop(to_drop_play,axis = 1, inplace= True)

### Testing team : player count

In [32]:
t = df_pstats[['player_id','player_team']]

In [46]:
t.drop_duplicates(inplace=True)

/var/folders/cp/9jtgx6ps6ldg63161fmh4z840000gn/T/ipykernel_10885/3295793208.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t.drop_duplicates(inplace=True)


In [47]:
t1 = t.sort_values(['player_id','player_team'],ascending=True).groupby('player_team').count().reset_index().sort_values(['player_id'],ascending=False)

In [48]:
t1

,player_team,player_id
2,1,218273
3,2,210786
4,3,134163
1,0,49824
5,4,36995
6,5,26537
7,6,21845
8,7,20172
9,8,4974
10,9,3887


### Testing iscomputer with player

In [50]:
t2 = df_pstats[['player_id','player_is_computer']]

In [51]:
t2.drop_duplicates(inplace=True)

/var/folders/cp/9jtgx6ps6ldg63161fmh4z840000gn/T/ipykernel_10885/186619806.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t2.drop_duplicates(inplace=True)


In [52]:
test2 = t2.sort_values(['player_id','player_is_computer'],ascending=True).groupby('player_is_computer').count().reset_index().sort_values(['player_id'],ascending=False)

In [53]:
test2

,player_is_computer,player_id
0,False,270420
1,True,13


### Testing game with featuremod

In [54]:
t_game = df_pstats[['game_id','game_featuremod_name']]

In [55]:
t_game.drop_duplicates(inplace=True)

/var/folders/cp/9jtgx6ps6ldg63161fmh4z840000gn/T/ipykernel_10885/2507246635.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t_game.drop_duplicates(inplace=True)


In [57]:
test3 = t_game.sort_values(['game_id','game_featuremod_name'],ascending=True).groupby('game_featuremod_name').count().reset_index().sort_values(['game_id'],ascending=False)

In [58]:
test3

,game_featuremod_name,game_id
4,FAF,7597873
1,Coop,1296793
9,Ladder1v1,1144443
11,Nomads,282513
14,Xtreme Wars,205318
5,FAF Beta Balance,105633
12,Phantom-X,66335
6,FAF Develop,42808
3,Equilibrium,26924
8,LABwars,12826


### Testing total games played by top rated players

In [59]:
t_top_players = df_pstats[['game_id','player_id','after_rating']]

In [60]:
t_top_players.drop_duplicates(inplace=True)

/var/folders/cp/9jtgx6ps6ldg63161fmh4z840000gn/T/ipykernel_10885/1446941319.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t_top_players.drop_duplicates(inplace=True)


In [61]:
t_top_players.head()

,game_id,player_id,after_rating
0,17405720,431007,99999999.0
1,17405711,96808,99999999.0
2,17405650,303409,159.7
3,17405650,436692,692.4
4,17405650,17943,1078.0


In [100]:
test4 = t_top_players.sort_values(['after_rating'],ascending=False).groupby(['after_rating']).agg({'game_id':['count']}).reset_index().sort_values(('game_id','count'),ascending=False)

In [102]:
test4[test4['after_rating']!= 99999999.0][:50]

,after_rating,game_id
,,count
11889,477.9,6402
6579,-53.1,5939
11839,472.9,2892
14151,704.1,2434
6564,-54.6,2410
15502,839.2,1815
16419,930.9,1671
14106,699.6,1652
10547,343.7,1613


In [96]:
# plt.bar(x=test4[('game_id','count')],height=test4.after_rating)

In [55]:
df_pstats.to_csv("player_stats_clean.csv",index = False)

### Data Cleaning Issue Command Stats file

In [56]:
df_issue = pd.read_csv('issue_stats.csv')

In [57]:
df_issue.shape

(1327457, 15)

In [58]:
df_issue.head()

,game_id,player_id,issue_apm,reclaim_ratio,overcharge_ratio,transport_ratio,structure_ratio,air_ratio,land_ratio,naval_ratio,t1_ratio,t2_ratio,t3_ratio,t4_ratio,defense_ratio
0,10000006,304205,33.300943,0.022657,0.0,0.000000,0.141092,0.017508,0.078270,0.0,0.122554,0.065911,0.019567,0.00103,0.008239
1,10000006,258198,46.119403,0.029873,0.0,0.004481,0.109037,0.011949,0.047050,0.0,0.103062,0.048544,0.006721,0.00000,0.052278
2,10000034,26737,20.049689,0.100372,0.0,0.000000,0.278810,0.014870,0.167286,0.0,0.364312,0.074349,0.000000,0.00000,0.018587
3,10000034,41066,19.076623,0.003831,0.0,0.000000,0.203065,0.000000,0.130268,0.0,0.283525,0.019157,0.000000,0.00000,0.015326
4,10000035,226514,43.486778,0.010135,0.0,0.011824,0.222973,0.025338,0.150338,0.0,0.285473,0.065878,0.000000,0.00000,0.010135


In [59]:
to_drop_stats = ['structure_ratio','air_ratio','land_ratio','naval_ratio','t1_ratio','t2_ratio','t3_ratio','t4_ratio','defense_ratio']
df_issue.drop(to_drop_stats,axis = 1, inplace= True)

In [60]:
df_issue.isnull().sum()

game_id               0
player_id             0
issue_apm           263
reclaim_ratio         0
overcharge_ratio      0
transport_ratio       0
dtype: int64

In [61]:
df_issue["issue_apm"].replace(to_replace=np.nan,value=280,inplace=True)

In [62]:
df_issue['issue_apm_cat'] = pd.cut(df_issue['issue_apm'], 11,include_lowest=False,right=False)

In [64]:
df_issue['issue_apm_cat'] = [str(x) for x in df_issue['issue_apm_cat']]

In [65]:
df_issue['issue_apm_cat'] = [f"Between {x.split(' ')[0].split('[')[1].split(',')[0]} and {x.split(' ')[1].split(')')[0]}" for x in df_issue['issue_apm_cat']]

In [66]:
df_issue["issue_apm_cat"].replace(to_replace='Between 254.545 and 280.28',value="Unknown",inplace=True)

In [67]:
df_issue['reclaim_ratio_cat'] = pd.cut(df_issue['reclaim_ratio'], bins=10,include_lowest=False,right=False)

In [68]:
df_issue['overcharge_ratio_cat'] = pd.cut(df_issue['overcharge_ratio'], bins=np.linspace(0.0,0.2,3),include_lowest=False,right=False)

In [69]:
df_issue['transport_ratio_cat'] = pd.cut(df_issue['transport_ratio'], bins=np.linspace(0.0,0.4,5),include_lowest=False,right=False)

In [70]:
max(df_issue['transport_ratio'])

0.3091603053435114

In [71]:
df_issue['reclaim_ratio_cat'] = [str(x) for x in df_issue['reclaim_ratio_cat']]

In [72]:
df_issue['overcharge_ratio_cat'] = [str(x) for x in df_issue['overcharge_ratio_cat']]

In [73]:
df_issue['transport_ratio_cat'] = [str(x) for x in df_issue['transport_ratio_cat']]

In [74]:
df_issue['reclaim_ratio_cat'] = [f"Between {x.split(' ')[0].split('[')[1].split(',')[0]} and {x.split(' ')[1].split(')')[0]}" for x in df_issue['reclaim_ratio_cat']]

In [75]:
df_issue['overcharge_ratio_cat'] = [f"Between {x.split(' ')[0].split('[')[1].split(',')[0]} and {x.split(' ')[1].split(')')[0]}" for x in df_issue['overcharge_ratio_cat']]

In [76]:
df_issue['transport_ratio_cat'] = [f"Between {x.split(' ')[0].split('[')[1].split(',')[0]} and {x.split(' ')[1].split(')')[0]}" for x in df_issue['transport_ratio_cat']]

In [77]:
df_issue.head()

,game_id,player_id,issue_apm,reclaim_ratio,overcharge_ratio,transport_ratio,issue_apm_cat,reclaim_ratio_cat,overcharge_ratio_cat,transport_ratio_cat
0,10000006,304205,33.300943,0.022657,0.0,0.000000,Between 25.455 and 50.909,Between 0.0 and 0.1,Between 0.0 and 0.1,Between 0.0 and 0.1
1,10000006,258198,46.119403,0.029873,0.0,0.004481,Between 25.455 and 50.909,Between 0.0 and 0.1,Between 0.0 and 0.1,Between 0.0 and 0.1
2,10000034,26737,20.049689,0.100372,0.0,0.000000,Between 6.57e-06 and 25.455,Between 0.1 and 0.2,Between 0.0 and 0.1,Between 0.0 and 0.1
3,10000034,41066,19.076623,0.003831,0.0,0.000000,Between 6.57e-06 and 25.455,Between 0.0 and 0.1,Between 0.0 and 0.1,Between 0.0 and 0.1
4,10000035,226514,43.486778,0.010135,0.0,0.011824,Between 25.455 and 50.909,Between 0.0 and 0.1,Between 0.0 and 0.1,Between 0.0 and 0.1


In [78]:
to_drop_stats = ['issue_apm','reclaim_ratio','overcharge_ratio','transport_ratio']
df_issue.drop(to_drop_stats,axis = 1, inplace= True)

In [95]:
df_issue['transport_ratio_cat'].unique()

array(['Between 0.0 and 0.1', 'Between 0.1 and 0.2',
       'Between 0.2 and 0.3', 'Between 0.3 and 0.4'], dtype=object)

In [79]:
df_issue.to_csv("issues_stats_clean.csv",index = False)

### Data Cleaning Command stats by month file

In [80]:
df_jan = pd.read_csv('issue_cmds.jan22.csv')

/var/folders/cp/9jtgx6ps6ldg63161fmh4z840000gn/T/ipykernel_9885/1507841470.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df_jan = pd.read_csv('issue_cmds.jan22.csv')


In [81]:
df_feb = pd.read_csv('issue_cmds.feb22.csv')

/var/folders/cp/9jtgx6ps6ldg63161fmh4z840000gn/T/ipykernel_9885/1678507369.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df_feb = pd.read_csv('issue_cmds.feb22.csv')


In [82]:
df_mar = pd.read_csv('issue_cmds.mar22.csv')

/var/folders/cp/9jtgx6ps6ldg63161fmh4z840000gn/T/ipykernel_9885/3559866882.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df_mar = pd.read_csv('issue_cmds.mar22.csv')


In [83]:
df_union_com = pd.concat([df_jan,df_feb,df_mar],ignore_index=True).drop_duplicates()

In [84]:
df_union_com.head()

,id,start_time,offset_ms,player,type,units_number,blueprint,position_x,position_y,command_name,faf_player_id,login,rating,faction,map_name,map_thumbnail,map_width,map_height
0,16128316,2022-01-14 20:00:38 UTC,1047300,1,issue,2,NaN,NaN,NaN,Stop,92766,Glassy,1577,Seraphim,Regor VI Highlands,https://content.faforever.com/maps/previews/la...,512,512
1,16178891,2022-01-21 17:44:31 UTC,927000,1,issue,1,NaN,NaN,NaN,Stop,64928,Morax,1750,UEF,White Fire - FAF version,https://content.faforever.com/maps/previews/la...,512,512
2,16256113,2022-01-31 11:47:12 UTC,1467100,1,issue,98,NaN,671.567932,271.30954,FormMove,108922,trandan,1772,Seraphim,The Bermuda Locket - FAF version,https://content.faforever.com/maps/previews/la...,1024,1024
3,16128700,2022-01-14 20:39:32 UTC,406700,1,issue,1,NaN,NaN,NaN,Stop,64928,Morax,1759,Aeon,White Fire - FAF version,https://content.faforever.com/maps/previews/la...,512,512
4,16031045,2022-01-01 17:56:41 UTC,659300,1,issue,1,NaN,NaN,NaN,Stop,366264,Keksyamba,636,UEF,Crossfire Canal - FAF version,https://content.faforever.com/maps/previews/la...,1024,1024


In [85]:
df_union_com.shape

(28175049, 18)

In [86]:
to_drop = ['offset_ms','player','position_x','position_y','map_thumbnail','map_width','map_height','login','rating','faction']
df_union_com.drop(to_drop,axis = 1, inplace= True)

In [87]:
df_union_com.isnull().sum()

id                      0
start_time              0
type                    0
units_number            0
blueprint        20345040
command_name            0
faf_player_id           0
map_name                0
dtype: int64

In [88]:
df_union_com["blueprint"].replace(to_replace=np.nan,value="unknown000",inplace=True)

In [89]:
df_union_com.rename(columns={'id':'game_id','type':'issue_type','blueprint':'blueprint_id','faf_player_id':'player_id'},inplace = True)

In [90]:
df_union_com.to_csv("janfebmar_combine_clean.csv",index = False)

In [96]:
df_union_com.head()

,game_id,start_time,issue_type,units_number,blueprint_id,command_name,player_id,map_name
0,16128316,2022-01-14 20:00:38 UTC,issue,2,unknown000,Stop,92766,Regor VI Highlands
1,16178891,2022-01-21 17:44:31 UTC,issue,1,unknown000,Stop,64928,White Fire - FAF version
2,16256113,2022-01-31 11:47:12 UTC,issue,98,unknown000,FormMove,108922,The Bermuda Locket - FAF version
3,16128700,2022-01-14 20:39:32 UTC,issue,1,unknown000,Stop,64928,White Fire - FAF version
4,16031045,2022-01-01 17:56:41 UTC,issue,1,unknown000,Stop,366264,Crossfire Canal - FAF version
